# Analyzing CIA Factbook Data Using SQL

In this project, we'll work with data from the [CIA World Factbook](https://www.cia.gov/the-world-factbook/), a compendium of statistics about all of the countries on Earth. The Factbook contains demographic information like the following:

* `population` — the global population.

* `population_growth` — the annual population growth rate, as a percentage.

* `area` — the total land and water area.

![](https://i.ibb.co/5kwfH4f/Web-capture-2-7-2021-235452-www-cia-gov.jpg)

## Connecting Database

In [1]:
%%capture
%load_ext sql
%sql sqlite:///factbook.db

'Connected: None@factbook.db'

In [2]:
%%sql
SELECT *
  FROM sqlite_master
 WHERE type = 'table';

Done.


type,name,tbl_name,rootpage,sql
table,sqlite_sequence,sqlite_sequence,3,"CREATE TABLE sqlite_sequence(name,seq)"
table,facts,facts,47,"CREATE TABLE ""facts"" (""id"" INTEGER PRIMARY KEY AUTOINCREMENT NOT NULL, ""code"" varchar(255) NOT NULL, ""name"" varchar(255) NOT NULL, ""area"" integer, ""area_land"" integer, ""area_water"" integer, ""population"" integer, ""population_growth"" float, ""birth_rate"" float, ""death_rate"" float, ""migration_rate"" float)"


In [3]:
%%sql
SELECT *
  FROM facts
 LIMIT 5;

Done.


id,code,name,area,area_land,area_water,population,population_growth,birth_rate,death_rate,migration_rate
1,af,Afghanistan,652230,652230,0,32564342,2.32,38.57,13.89,1.51
2,al,Albania,28748,27398,1350,3029278,0.3,12.92,6.58,3.3
3,ag,Algeria,2381741,2381741,0,39542166,1.84,23.67,4.31,0.92
4,an,Andorra,468,468,0,85580,0.12,8.13,6.96,0.0
5,ao,Angola,1246700,1246700,0,19625353,2.78,38.78,11.49,0.46


Here are the descriptions for some of the columns:

* **name** — the name of the country.


* **area** — the country's total area (both land and water).


* **area_land** — the country's land area in square kilometers.


* **area_water** — the country's waterarea in square kilometers.


* **population** — the country's population.


* **population_growth** — the country's population growth as a percentage.


* **birth_rate** — the country's birth rate, or the number of births per year per 1,000 people.


* **death_rate** — the country's death rate, or the number of death per year per 1,000 people.


* **migration_rate** - the country's migration rate, or change in population due to migration per 1,000 people.

## Summary Statistics

In [4]:
%%sql

SELECT MIN(population)
      ,MAX(population)
      ,MIN(population_growth)
      ,MAX(population_growth)
    
 FROM facts;

Done.


MIN(population),MAX(population),MIN(population_growth),MAX(population_growth)
0,7256490011,0.0,4.02


In [5]:
%%sql

SELECT name
  FROM facts
 WHERE population = 0;

Done.


name
Antarctica


In [6]:
%%sql

SELECT name
  FROM facts
 WHERE population = 7256490011;

Done.


name
World


> * **Antartica** has minimum population i.e, 0


> * **World**  population is 7256490011

In [7]:
%%sql

SELECT MIN(population)
      ,MAX(population)
      ,MIN(population_growth)
      ,MAX(population_growth)
  
  FROM facts
  WHERE name != 'World';


Done.


MIN(population),MAX(population),MIN(population_growth),MAX(population_growth)
0,1367485388,0.0,4.02


## Average Population & Area

In [8]:
%%sql

SELECT AVG(population),AVG(area)
  FROM facts;

Done.


AVG(population),AVG(area)
62094928.32231405,555093.546184739


In [9]:
%%sql

SELECT name
  FROM facts
 WHERE (population > (SELECT AVG(population)
                       FROM facts))
          &
        (area < (SELECT AVG(area)
                  FROM facts));

Done.


name
Bangladesh
Germany
Japan
Philippines
Thailand
United Kingdom
Vietnam


## Which country has the most people? Which country has the highest growth rate?

In [10]:
%%sql

SELECT name, 
       MAX(population)

  FROM facts
  WHERE name != 'World';

Done.


name,MAX(population)
China,1367485388


In [11]:
%%sql

SELECT name, 
       MAX(population_growth)

  FROM facts
  WHERE name != 'World';

Done.


name,MAX(population_growth)
South Sudan,4.02


> * **China** has maximum population which is 136.74 Cr .
>
>
> * **South Sudan** has maximum population growth rate which is 4.02% .

## Which countries have the highest ratios of water to land? Which countries have more water than land?

In [16]:
%%sql

SELECT name, 
       CAST(area_water AS FLOAT)/area_land AS water_land_ratio
  FROM facts
 WHERE water_land_ratio > 1;

Done.


name,water_land_ratio
British Indian Ocean Territory,905.6666666666666
Virgin Islands,4.520231213872832


>* There are two country with more water area than land.
>
>
>* **British Indian Ocean Territory** has highest water-land area ratio.
>
>
>* **Virgin Islands** is other country with more than 1 water-land ratio.

## Which countries will add the most people to their populations next year?

In [20]:
%%sql

SELECT name, 
      (population * (population_growth/100)) pop_incr
  
  FROM facts
 WHERE name != 'World'
 ORDER BY pop_incr DESC
  LIMIT 1;

Done.


name,pop_incr
India,15270686.124799998


> * **India** will add most people to population next year i.e, 1.53 Cr

## Which countries have a higher death rate than birth rate?

In [29]:
%%sql

SELECT name,
       birth_rate,
       death_rate,
       death_rate - birth_rate AS diff
        
  FROM facts
 WHERE death_rate > birth_rate
 ORDER BY death_rate DESC;

Done.


name,birth_rate,death_rate,diff
Ukraine,10.72,14.46,3.74
Bulgaria,8.92,14.44,5.52
Latvia,10.0,14.31,4.3100000000000005
Lithuania,10.1,14.27,4.17
Russia,11.6,13.69,2.09
Serbia,9.08,13.66,4.58
Belarus,10.7,13.36,2.66
Hungary,9.16,12.73,3.5700000000000003
Moldova,12.0,12.59,0.5899999999999999
Estonia,10.51,12.4,1.8900000000000006


In [28]:
%%sql

SELECT COUNT(name)     
  FROM facts
 WHERE death_rate > birth_rate;

Done.


COUNT(name)
24


> * Total 24 countries have higher death rate than birth rate.
>
>
> * **Ukraine** has greatest death rate in the list i.e., 14.46%.
>
>
>* **Bulgaria** has greatest decrease in net population i.e, 5.52%
>
>
> * Out of 24 countries, only two countries are not from Europe i.e, **Saint Pierre and Miquelon** & **Japan** .
>
>
> Rest 22 countrie are from **EUROPE**. This shows that there is decrese in population in European countries which is a food for thought!

## Which countries have the highest population/area ratio, and how does it compare to list we found in the previous screen?

In [31]:
%%sql

SELECT name,
       population/area AS popl_area_ratio
  
  FROM facts
 ORDER BY popl_area_ratio DESC
  LIMIT 5;


Done.


name,popl_area_ratio
Macau,21168
Monaco,15267
Singapore,8141
Hong Kong,6445
Gaza Strip,5191


> *  Among top 5 country with highest population area ration there is only one    country which has higher death rate than birth rate i.e, **Monaco** 

## Further Exploring Population Statistics

In [36]:
%%sql

SELECT name,
       population
 
  FROM facts
  WHERE population != 'None'
 ORDER BY population
 LIMIT 5;

Done.


name,population
Antarctica,0
Pitcairn Islands,48
Cocos (Keeling) Islands,596
Holy See (Vatican City),842
Niue,1190


In [46]:
%%sql

SELECT name,
       population,
       CAST(population AS FLOAT)/area_land AS popl_land_ratio
 
  FROM facts
  WHERE (population != 'None')
    AND (name != 'World' and name != 'European Union')
  ORDER BY population DESC
 LIMIT 5;

Done.


name,population,popl_land_ratio
China,1367485388,146.62505594328366
India,1251695584,420.9937208919838
United States,321368864,35.07640870965904
Indonesia,255993674,141.31047395931373
Brazil,204259812,24.43842912418313


> * **Antartica** has least population which is **0** . It amy be due to fact that it has harsh cold waether condition throughout the year.
>
>
> * Ignoring Antartica, **Pitcairn Islands** is the least populated entity with strength of **48**. 
>
>
> * **China** is most populated country i.e, **136.74 Cr**.
>
>
>* **India** is second most populated country, but has highest population density among top 5 populous countries which is a matter of concern.


## Exporing Migration Statistics

In [54]:
%%sql

SELECT name,migration_rate
  FROM facts
  WHERE migration_rate > 0
  ORDER BY migration_rate DESC;

Done.


name,migration_rate
Qatar,22.39
American Samoa,21.13
"Micronesia, Federated States of",20.93
Syria,19.79
Tonga,17.84
British Virgin Islands,17.28
Luxembourg,17.16
Cayman Islands,14.4
Singapore,14.05
Nauru,13.63


In [59]:
%%sql

SELECT count(name)
  FROM facts
  WHERE migration_rate > 0
    AND (name != 'World' and name != 'European Union')
  ORDER BY migration_rate DESC;

Done.


count(name)
187


In [56]:
%%sql

SELECT name,migration_rate
  FROM facts
  WHERE migration_rate < 0
  ORDER BY migration_rate;

Done.


name,migration_rate


In [61]:
%%sql

SELECT name,migration_rate
 FROM facts;

Done.


name,migration_rate
Afghanistan,1.51
Albania,3.3
Algeria,0.92
Andorra,0.0
Angola,0.46
Antigua and Barbuda,2.21
Argentina,0.0
Armenia,5.8
Australia,5.65
Austria,5.56


>*  **migration_rate** column has only non-negative values. So, this is ambiguous as many countries have negative migration rate in reality.
>
>
>
>*  **Quatar** has highest migration rate which is 	**22.39%** .